# Data extraction from a company's bills

The data is in Persian; thus, the characters are different from English. So that, I had to use Image processing.
* First, the ROI calibrated to encompass the desired data.
* Secondly, text data was detected from tables.
* Ultimately, extracted data grasped as pandas dataFrame.

In some pages, tables could not be detected right. So, I tried to extract them as text and then convert them into DataFrames.

The following code is the gist of what I utilized to attain data.

In [ ]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import pandas as pd

pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Define the coordinates of the area you want to crop in PDF points (x0, y0, x1, y1)
crop_box = (35, 150, 400, 250)

# Step 1: Convert PDF pages to images
pages = convert_from_path("some_bills.pdf", dpi=300)
text_data = []
all_tables = []  # Store manually parsed tables for further analysis

with pdfplumber.open("some_bills.pdf") as pdf:
    for page_num, (page_image, pdf_page) in enumerate(zip(pages, pdf.pages)):
        # Get the dimensions of the PDF page and image for scaling
        pdf_width, pdf_height = pdf_page.width, pdf_page.height
        img_width, img_height = page_image.size
        
        # Calculate scaling factors
        scale_x = img_width / pdf_width
        scale_y = img_height / pdf_height
        
        # Scale the crop box coordinates to match the image dimensions
        scaled_crop_box = (
            int(crop_box[0] * scale_x),
            int(crop_box[1] * scale_y),
            int(crop_box[2] * scale_x),
            int(crop_box[3] * scale_y)
        )
        
        # Step 2: Crop the image to the scaled area and perform OCR
        cropped_image = page_image.crop(scaled_crop_box)
        text = pytesseract.image_to_string(cropped_image, lang="fas", config="--psm 6")
        text_data.append(text)

        # Step 3: Try to manually parse the extracted text as a table
        if text:
            # Split lines and attempt to format as a table
            rows = [line.split() for line in text.splitlines() if line.strip()]  # Split by whitespace
            if rows:
                df = pd.DataFrame(rows)  # Convert to DataFrame for each page's table
                all_tables.append(df)
                print(f"Table-like data extracted from cropped area on page {page_num + 1}:")
                print(df)

# Print the OCR results
print("Text extracted from the cropped area:", text_data)

# Combine extracted tables if any were successfully parsed
if all_tables:
    combined_df = pd.concat(all_tables, ignore_index=True)
    print("\nCombined table data for analysis:")
    print(combined_df)